# Árboles de decisión
### Implementación a mano y uso de librería en sklearn

<!-- ![a](https://files.ai-pool.com/a/aba6fb4dcd4c3d01372085631f47d122.png) -->
<img src="https://files.ai-pool.com/a/aba6fb4dcd4c3d01372085631f47d122.png" width="40%" />

Los árboles de decisión son una herramienta de clasificación que utiliza una estructura de árbol para su toma de decisiones, en particular, utiliza a un modelo de condiciones estructuradas de forma jerárquica para determinar la etiqueta asociada a cada dato. Para construir un árbol de decisión deberemos indicar mediante una métrica de separabilidad de los datos qué características nos son más útiles para llevar a cabo este proceso.

In [2]:
# Función que nos permite calcular el logaritmo de base de 2 de un número
from math import log2
import pandas as pd

# Importamos DecisionTree de su implementación en sklearn para construir el Random Forest
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text

Crearemos un set de datos que nos ayudarán a predecir si un partido de Tenis se jugará o no dependiendo de las condiciones climáticas


| Outlook | Temperature | Humidity | Wind | Plays |
|:-------:|:-----------:|:--------:|:----:| :---: |
|  Sunny  |     Hot     |   High   | Weak |  No   |
|  Rain   |     Mild    |  Normal  | Weak |  Yes  |
| ... | ... | ... | ... | ... |

In [1]:
random_state = 0

# Definición del set de datos y los nombres de sus atributos
headers = ['Outlook', 'Temperature', 'Humidity', 'Wind', 'Plays']

data=[['Sunny','Hot','High','Weak','No'],
      ['Sunny','Hot','High','Strong','No'],
      ['Overcast','Hot','High','Weak','Yes'],
      ['Rain','Mild','High','Weak','Yes'],
      ['Rain','Cool','Normal','Weak','Yes'],
      ['Rain','Cool','Normal','Strong','No'],
      ['Overcast','Cool','Normal','Strong','Yes'],
      ['Sunny','Mild','High','Weak','No'],
      ['Sunny','Cool','Normal','Weak','Yes'],
      ['Rain','Mild','Normal','Weak','Yes'],
      ['Sunny','Mild','Normal','Strong','Yes'],
      ['Overcast','Mild','High','Strong','Yes'],
      ['Overcast','Hot','Normal','Weak','Yes'],
      ['Rain','Mild','High','Strong','No']]

Definiremos el nodo de un árbol, que contiene la columna que tiene asociada y el valor del atributo que llevó a este nodo

In [ ]:
class DecisionNode:
    def __init__(self, col = -1, value = '', branches = None, results = None):
        self.col=col
        self.value = value
        self.results=results
        self.branches = branches

Definiremos una función que nos permita determinar la cantidad de datos para cada una de las clases:

In [ ]:
# Conteo de registros pertenecientes a cada clase
def unique_counts(rows):
    results = {}
    for row in rows:
        r = row[-1]
        if r not in results: 
            results[r] = 0
        results[r] += 1
    return results

unique_counts(data)

{'No': 5, 'Yes': 9}

Ahora, definiremos una serie de funciones que nos permitirá trabajar con este set de datos para crear un árbol de decisión

In [ ]:
# La entropía es una métrica que mide la capacidad de separación sobre los datos de una característica

def entropy(rows):
    # We find a label to measure the degree of separation with
    results = unique_counts(rows)
    
    # Now calculate the entropy
    ent = 0.0
    for r in results.keys():
        p = float(results[r]) / len(rows) 
        ent = ent - p * log2(p)
    return ent

entropy(data)

0.9402859586706311

In [ ]:
# División del set de datos de acuerdo a los valores de un atributo
# recibe una lista de listas en el argumento "rows", cada una correspondiente a una fila de los datos
# recibe una índice en el argumento "column", que indica bajo qué columna separaremos los datos

def divide_set(rows, column):
    sets = dict()
    for row in rows:
        if row[column] not in sets.keys():
            sets[row[column]] = []
        sets[row[column]].append(row)
    return sets

divide_set(data, 0)

{'Sunny': [['Sunny', 'Hot', 'High', 'Weak', 'No'],
  ['Sunny', 'Hot', 'High', 'Strong', 'No'],
  ['Sunny', 'Mild', 'High', 'Weak', 'No'],
  ['Sunny', 'Cool', 'Normal', 'Weak', 'Yes'],
  ['Sunny', 'Mild', 'Normal', 'Strong', 'Yes']],
 'Overcast': [['Overcast', 'Hot', 'High', 'Weak', 'Yes'],
  ['Overcast', 'Cool', 'Normal', 'Strong', 'Yes'],
  ['Overcast', 'Mild', 'High', 'Strong', 'Yes'],
  ['Overcast', 'Hot', 'Normal', 'Weak', 'Yes']],
 'Rain': [['Rain', 'Mild', 'High', 'Weak', 'Yes'],
  ['Rain', 'Cool', 'Normal', 'Weak', 'Yes'],
  ['Rain', 'Cool', 'Normal', 'Strong', 'No'],
  ['Rain', 'Mild', 'Normal', 'Weak', 'Yes'],
  ['Rain', 'Mild', 'High', 'Strong', 'No']]}

In [ ]:
# Construcción del árbol usando ganancia de información

def build_tree(rows, headers, value = ''):
    # En caso de tener un árbol vacío, retornamos un nodo vacío
    if len(rows) == 0: 
        return DecisionNode(value = value)
    
    # En caso de tener una sola columna, creamos un único separador en base a ellas
    elif len(set([row[-1] for row in rows])) == 1:
        return DecisionNode(value = value, results = unique_counts(rows))
    
    # Si tenemos múltiples columnas llevamos a cabo el siguiente proceso:
    current_score = entropy(rows)
    best_gain = 0.0
    best_criteria = None
    best_sets = None
    column_count = len(rows[0]) - 1

    for col in range(0, column_count):
        # Dividimos el set de datos acorde a los valores de la columna "col"
        sets = divide_set(rows, col)
        gain = current_score

        # Para cada valor único de la columna "col", calculamos qué fracción de las filas separa,
        # luego, multiplicamos este factor por la entropía de la característica sobre la etiqueta a predecir
        for key in sets.keys():
            s = float(len(sets[key])) / len(rows)
            gain -= s * entropy(sets[key])

        # Almacenamos aquella columna con mejor ganancia hasta el momento
        if gain > best_gain:
            best_gain = gain
            best_criteria = headers[col]
            best_sets = sets

    branches = []
    # Cara cada valor de las mejor característica, crearemos un sub-árbol que lo separe hasta llegar a una sola columna
    for s in best_sets:
        branches.append(build_tree(best_sets[s], headers, s))

    # Retornamos el nodo inicial del árbol, con la primera mejor caracaterística de separador
    return DecisionNode(col = best_criteria, value = value, branches = branches)

In [ ]:
# Función para imprimir el árbol de decisión construido
def print_tree(tree, indent = ''):
    if tree.results != None:
        print(indent + str(tree.results))
    else:
        if tree.value == '':
            print(indent + tree.col + "?")
        for branch in tree.branches:
            to_print = indent + '  ' + branch.value + " -> "
            if branch.col != -1:
                print(to_print + branch.col + "?")
                print_tree(branch, ' '*len(to_print))
            else:
                print(to_print + str(branch.results))

In [ ]:
print_tree(build_tree(data, headers))

Outlook?
  Sunny -> Humidity?
             High -> {'No': 3}
             Normal -> {'Yes': 2}
  Overcast -> {'Yes': 4}
  Rain -> Wind?
            Weak -> {'Yes': 3}
            Strong -> {'No': 2}


Ahora, utilizamos la implementación de ``sklearn`` de ``DecisionTreeClassifier``

In [ ]:
# Creamos un DataFrame con nuestros datos
df = pd.DataFrame(data, columns = headers)

# Pasamos de valores en texto a valores numéricos:
for column_name in df.columns:
    df[column_name] = df[column_name].astype('category').cat.codes

# Separamos entre matriz de características y vector de etiquetas
X = df.drop("Plays", axis=1)
y = df["Plays"]

In [ ]:
tree = DecisionTreeClassifier(criterion = "entropy", random_state = random_state)
tree.fit(X, y)

print(export_text(tree))

|--- feature_0 <= 0.50
|   |--- class: 1
|--- feature_0 >  0.50
|   |--- feature_2 <= 0.50
|   |   |--- feature_0 <= 1.50
|   |   |   |--- feature_3 <= 0.50
|   |   |   |   |--- class: 0
|   |   |   |--- feature_3 >  0.50
|   |   |   |   |--- class: 1
|   |   |--- feature_0 >  1.50
|   |   |   |--- class: 0
|   |--- feature_2 >  0.50
|   |   |--- feature_3 <= 0.50
|   |   |   |--- feature_1 <= 1.00
|   |   |   |   |--- class: 0
|   |   |   |--- feature_1 >  1.00
|   |   |   |   |--- class: 1
|   |   |--- feature_3 >  0.50
|   |   |   |--- class: 1



## Conclusión

- Los árboles de decisión utilizan una estructura jerárquica para su construcción y toma de decisiones, priorizando características más informativas en bases a una determinada métrica.
- La entropía es un método de determinar la capacidad informativa de una característica, esta busca determinar qué tanto separan cada uno de sus valores únicos las etiquetas que luego buscamos predecir.
- ``sklearn`` provee su propio método para implementar árboles de decisión (``DecisionTreeClassifier``), para su uso deberemos trabajar los datos en un formato numérico y en forma de matrices o DataFrames.